In [2]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.datasets import cifar10
import keras.preprocessing.image
import numpy as np
import random
import keras
import scipy
import cv2
from scipy.misc import imresize

# load the data
#(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#x_train, y_train = load_data()
# Convert class vectors to binary class matrices.
#y_train = keras.utils.to_categorical(y_train, 10)
#y_test = keras.utils.to_categorical(y_test, 10)

#y_train = np.squeeze(y_train)
print('data loaded')

c:\users\jesper\appdata\local\conda\conda\envs\tensorflowv3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


data loaded


In [3]:
# Examples
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train[0:1000]
y_train = y_train[0:1000]
x_test = x_test[0:1000]
y_test = y_test[0:1000]


x_train = [imresize(image, (139,139)) for image in x_train]
x_test = [imresize(image, (139,139)) for image in x_test]

num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


c:\users\jesper\appdata\local\conda\conda\envs\tensorflowv3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  # Remove the CWD from sys.path while we load stuff.
c:\users\jesper\appdata\local\conda\conda\envs\tensorflowv3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  # This is added back by InteractiveShellApp.init_path()


In [4]:
datagen = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=0,
    width_shift_range=0,
    height_shift_range=0,
    horizontal_flip=False
    )

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)

In [5]:
print(x_train[1].shape)

(139, 139, 3)


In [67]:
def load_data():
    (_, _), (x_train, y_train) = cifar10.load_data()
    x_train = x_train[:100]
    y_train = y_train[:100]
    print(x_train.shape)

    data_upscaled = np.zeros((100, 3, 299, 299))

    for i, img in enumerate(x_train):
        im = img.transpose((1, 2, 0))
        large_img = cv2.resize(
            im, dsize=(299, 299), interpolation=cv2.INTER_CUBIC)
        data_upscaled[i] = large_img.transpose((2, 0, 1))

    y_train = to_categorical(y_train, 10)

    return data_upscaled, y_train

In [6]:
# this could also be the output a different Keras model or layer
#input_tensor = Input(shape=(32, 32, 3))  # this assumes K.image_data_format() == 'channels_last'

# create the base pre-trained model
#base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(139, 139, 3))
#base_model = InceptionV3(weights='imagenet', include_top=False)
print('model loaded')

model loaded


In [7]:

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(10, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [8]:

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
#model.fit_generator(...)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.


In [9]:

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True



0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [10]:
def generator(features, labels, batch_size):
 # Create empty arrays to contain batch of features and labels#
 batch_features = np.zeros((batch_size, 32, 32, 3))
 batch_labels = np.zeros((batch_size,1))
 while True:
   for i in range(batch_size):
     # choose random index in features
     index= random.randint(0,features.shape[0])
     batch_features[i] = features[index]
     batch_labels[i] = labels[index]
   yield batch_features, batch_labels
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

    
# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                        steps_per_epoch=len(x_train) / 32, epochs=epochs)

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
#model.fit(x_train, y_train, batch_size=32, epochs=10,validation_data=(x_test, y_test))
#model.fit_generator(generator(x_train, y_train, 32),samples_per_epoch=50,nb_epoch=10)

AttributeError: 'list' object has no attribute 'shape'

In [11]:

batch_size = 32
num_classes = 10
epochs = 100
#x_train = x_train.astype('float32')
#x_test = x_test.astype('float32')
#x_train /= 255
#x_test /= 255
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 1000 arrays: [array([[[ 59,  62,  63],
        [ 59,  62,  63],
        [ 58,  61,  62],
        ...,
        [148, 124, 103],
        [148, 124, 103],
        [148, 124, 103]],

       [[ 59,  62,  63],
        [...